# LangChain Agent with Qiskit Code Assistant MCP Server

This notebook demonstrates how to create an AI agent using LangChain that connects to the **qiskit-code-assistant-mcp-server** via the Model Context Protocol (MCP).

The agent can interact with Qiskit Code Assistant to:
- Generate quantum code completions
- Answer questions about Qiskit and quantum computing (RAG mode)
- Accept model disclaimers
- Track completion acceptance

## Architecture

```
┌─────────────┐     MCP Protocol     ┌──────────────────────────────────┐
│  LangChain  │ ◄──────────────────► │ qiskit-code-assistant-mcp-server │
│    Agent    │                      │                                  │
└─────────────┘                      │  ┌────────────────────────────┐  │
                                     │  │  Qiskit Code Assistant API │  │
                                     │  └────────────────────────────┘  │
                                     │               │                  │
                                     └───────────────│──────────────────┘
                                                     ▼
                                            ┌─────────────────┐
                                            │  IBM Quantum    │
                                            │  Code Assistant │
                                            └─────────────────┘
```

## Setup

### 1. Install Dependencies

Run these commands in your terminal:

```bash
# Install the MCP server with examples dependencies
pip install qiskit-code-assistant-mcp-server[examples]

# Install your preferred LLM provider (choose one):
pip install langchain-openai       # For OpenAI
pip install langchain-anthropic    # For Anthropic Claude
pip install langchain-google-genai # For Google Gemini
pip install langchain-ollama       # For local Ollama
pip install langchain-ibm          # For IBM Watsonx
```

### 2. Configure Environment Variables

Set your IBM Quantum token for the Qiskit Code Assistant.

You can either:
- Set them in a `.env` file in this directory
- Set them as environment variables
- Enter them in the cell below

In [1]:
import os

from dotenv import load_dotenv

# LangChain imports
from langchain.agents import create_agent
from langchain_core.messages import HumanMessage
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain_mcp_adapters.tools import load_mcp_tools


# Load from .env file if it exists
load_dotenv()

# Or set directly (uncomment and fill in):
# os.environ["QISKIT_IBM_TOKEN"] = "your-ibm-quantum-token"

# Set your LLM provider API key (uncomment the one you're using):
# os.environ["OPENAI_API_KEY"] = "your-openai-api-key"
# os.environ["ANTHROPIC_API_KEY"] = "your-anthropic-api-key"
# os.environ["GOOGLE_API_KEY"] = "your-google-api-key"

# Verify configuration
print("Configuration status:")
print(f"  QISKIT_IBM_TOKEN: {'✓ Set' if os.getenv('QISKIT_IBM_TOKEN') else '✗ Not set'}")

Configuration status:
  QISKIT_IBM_TOKEN: ✓ Set


## Choose Your LLM Provider

Run **one** of the following cells based on your preferred LLM provider:

In [ ]:
# Option 1: OpenAI
from langchain_openai import ChatOpenAI


llm = ChatOpenAI(model="gpt-4o", temperature=0)
print("Using OpenAI GPT-4o")

In [ ]:
# Option 2: Anthropic Claude
from langchain_anthropic import ChatAnthropic


llm = ChatAnthropic(model="claude-sonnet-4-20250514", temperature=0)
print("Using Anthropic Claude Sonnet")

In [ ]:
# Option 3: Google Gemini
from langchain_google_genai import ChatGoogleGenerativeAI


llm = ChatGoogleGenerativeAI(model="gemini-2.5-pro", temperature=0)
print("Using Google Gemini Pro")

In [2]:
# Option 4: Local Ollama (no API key needed)
from langchain_ollama import ChatOllama


llm = ChatOllama(model="llama3.2", temperature=0)
print("Using local Ollama with Llama 3.2")

Using local Ollama with Llama 3.2


In [ ]:
# Option 5: IBM Watsonx
from langchain_ibm import ChatWatsonx


llm = ChatWatsonx(
    model_id="ibm/granite-3-8b-instruct",
    url=os.getenv("WATSONX_URL", "https://us-south.ml.cloud.ibm.com"),
    project_id=os.getenv("WATSONX_PROJECT_ID"),
    params={"temperature": 0, "max_tokens": 4096},
)
print("Using IBM Watsonx Granite")

## Define the System Prompt

This prompt tells the agent what it can do and how to behave:

In [3]:
SYSTEM_PROMPT = """You are a helpful quantum computing coding assistant with access to Qiskit Code Assistant
through the MCP server.

You can help users:
- Generate quantum code using Qiskit (get_completion tool)
- Answer questions about Qiskit and quantum computing concepts (get_rag_completion tool)
- Accept model disclaimers when required (accept_model_disclaimer tool)
- Track completion acceptance for feedback (accept_completion tool)

When generating code:
- Use the get_completion tool to generate Qiskit code based on user prompts
- Provide clear explanations of the generated code
- Suggest improvements or alternatives when appropriate

When answering questions:
- Use the get_rag_completion tool for conceptual questions about quantum computing
- Provide accurate and educational explanations
- Reference Qiskit documentation when relevant

If you encounter a disclaimer requirement, use accept_model_disclaimer to accept it.
After using a completion, use accept_completion to provide feedback."""

## Create the MCP Client

This configures the connection to the qiskit-code-assistant-mcp-server:

In [4]:
def get_mcp_client():
    """Create and return an MCP client configured for the Qiskit Code Assistant server."""
    return MultiServerMCPClient(
        {
            "qiskit-code-assistant": {
                "transport": "stdio",
                "command": "qiskit-code-assistant-mcp-server",
                "args": [],
                "env": {
                    "QISKIT_IBM_TOKEN": os.getenv("QISKIT_IBM_TOKEN", ""),
                },
            }
        }
    )

## Create the Agent

Now we'll create a function that sets up the agent with a persistent MCP session.

Using a persistent session is important because it:
- Keeps a single MCP server process running
- Reuses the Qiskit Code Assistant connection
- Makes tool calls much faster

In [5]:
async def create_agent_with_session(session):
    """Create a LangChain agent using an existing MCP session."""
    # Load tools from the existing session
    tools = await load_mcp_tools(session)
    print(f"Loaded {len(tools)} tools from MCP server:")
    for tool in tools:
        print(f"  - {tool.name}")

    # Create the agent using LangChain's create_agent
    agent = create_agent(llm, tools, system_prompt=SYSTEM_PROMPT)
    return agent

## Helper Function to Run Queries

This function sends a query to the agent and returns the response:

In [6]:
async def ask_agent(agent, query: str) -> str:
    """Send a query to the agent and return the response."""
    result = await agent.ainvoke({"messages": [HumanMessage(content=query)]})
    messages = result.get("messages", [])
    if messages:
        return messages[-1].content
    return "No response generated."

## Run the Agent

Now let's create the agent and ask it some questions!

The following cell starts the MCP server, creates the agent, and keeps the session open for multiple queries:

In [7]:
# Create MCP client and start a persistent session
mcp_client = get_mcp_client()

print("Starting MCP server and creating agent...")
print("(This may take a few seconds on first run)\n")

Starting MCP server and creating agent...
(This may take a few seconds on first run)



### Example 1: Generate a Bell State Circuit

Let's ask the agent to generate code for a Bell state:

In [8]:
async with mcp_client.session("qiskit-code-assistant") as session:
    agent = await create_agent_with_session(session)

    response = await ask_agent(agent, "Write a quantum circuit that creates a Bell state")
    print(response)

Loaded 4 tools from MCP server:
  - accept_model_disclaimer_tool
  - get_completion_tool
  - get_rag_completion_tool
  - accept_completion_tool
The Qiskit program that creates a Bell state using the `EfficientSU2` ansatz from Qiskit's circuit library is as follows:

```python
from qiskit.circuit.library import EfficientSU2
from qiskit.quantum_info import SparsePauliOp
from qiskit.primitives import EstimatorV2 as Estimator
from qiskit_ibm_runtime import QiskitRuntimeService

# Create a 2-qubit circuit with entanglement
ansatz = EfficientSU2(2, entanglement="full", reps=1)

# Define the observable for the Bell state <ZI+IZ>
observable = SparsePauliOp.from_list([("ZI", 1), ("IZ", 1)])

# Initialize the estimator
service = QiskitRuntimeService()
backend = service.least_busy(operational=True, simulator=False)
estimator = Estimator(backend)

# Run the estimator
job = estimator.run([(ansatz, observable)])
result = job.result()

# Print the result
print(f"Expectation value: {result[0].data.evs

### Example 2: Ask a Quantum Computing Question

Use RAG mode to get answers about quantum computing concepts:

In [9]:
async with mcp_client.session("qiskit-code-assistant") as session:
    agent = await create_agent_with_session(session)

    response = await ask_agent(agent, "What is quantum entanglement and why is it important?")
    print(response)

Loaded 4 tools from MCP server:
  - accept_model_disclaimer_tool
  - get_completion_tool
  - get_rag_completion_tool
  - accept_completion_tool
Quantum entanglement is a fundamental phenomenon in quantum mechanics where the quantum states of two or more objects become interconnected. This means the state of one object instantly influences the state of the other, regardless of the distance between them.

The importance of quantum entanglement lies in its applications:

1. Quantum Teleportation: Allows the transfer of quantum information from one location to another using entangled particles and classical communication.
2. Superdense Coding: Enables the transmission of two classical bits of information using one qubit and an entangled pair.
3. Quantum Computing: Entanglement is crucial for quantum algorithms, enabling parallelism and exponential speedup for certain problems.
4. Quantum Cryptography: Ensures secure communication through protocols like Quantum Key Distribution (QKD), which

### Example 3: Generate a VQE Circuit

Let's generate more complex quantum code:

In [10]:
async with mcp_client.session("qiskit-code-assistant") as session:
    agent = await create_agent_with_session(session)

    response = await ask_agent(
        agent, "Generate code to set up a simple VQE algorithm for finding the ground state energy"
    )
    print(response)

Loaded 4 tools from MCP server:
  - accept_model_disclaimer_tool
  - get_completion_tool
  - get_rag_completion_tool
  - accept_completion_tool
The completed code is:

```python
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import TwoLocal
from qiskit.primitives import EstimatorV2 as Estimator
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import SPSA

# 1. Define the Hamiltonian
hamiltonian = SparsePauliOp.from_list(
    [
        ("II", -1.052373245772859),
        ("IZ", 0.39793742484318045),
        ("ZI", -0.39793742484318045),
        ("ZZ", -0.01128010425623538),
        ("XX", 0.18093119978423156),
    ]
)

# 2. Set up the ansatz
ansatz = TwoLocal(
    num_qubits=2,
    rotation_blocks="ry",
    entanglement_blocks="cz",
    entanglement="full",
    reps=3,
)

# 3. Configure the estimator
estimator = Estimator()

# 4. Set up the optimizer
optimizer = SPSA(maxiter=100)

# 5. Run the VQE
vqe = VQE(estimator=estimator, ansatz=ansatz

### Example 4: Explain Qiskit Concepts

Ask about Qiskit-specific features:

In [11]:
async with mcp_client.session("qiskit-code-assistant") as session:
    agent = await create_agent_with_session(session)

    response = await ask_agent(agent, "How does Qiskit's transpiler work?")
    print(response)

Loaded 4 tools from MCP server:
  - accept_model_disclaimer_tool
  - get_completion_tool
  - get_rag_completion_tool
  - accept_completion_tool
The Qiskit transpiler is a remote transpilation service that optimizes quantum circuits for specific hardware. It is part of the Qiskit SDK and provides a way to optimize circuits before executing them on IBM Quantum hardware.

The transpiler works by analyzing the circuit and identifying opportunities to reduce the number of qubits, gates, and other resources required to execute the circuit. This can help improve the performance and efficiency of the circuit, as well as reduce the risk of errors due to noise or other factors in the quantum system.

The Qiskit transpiler service is available on the cloud and can be used to optimize circuits for a variety of hardware platforms, including IBM Quantum's quantum computers and other third-party systems. It also provides a way to test and validate circuits before executing them on actual hardware.

I

### Example 5: Interactive Chat

Run this cell to have an interactive conversation with the agent:

In [12]:
async with mcp_client.session("qiskit-code-assistant") as session:
    agent = await create_agent_with_session(session)
    print("Agent ready! Type your questions below.")
    print("Enter 'quit' to stop.\n")

    while True:
        try:
            query = input("You: ").strip()
            if not query:
                continue
            if query.lower() in ["quit", "exit", "q"]:
                print("Goodbye!")
                break

            response = await ask_agent(agent, query)
            print(f"\nAssistant: {response}\n")
        except KeyboardInterrupt:
            print("\nGoodbye!")
            break

Loaded 4 tools from MCP server:
  - accept_model_disclaimer_tool
  - get_completion_tool
  - get_rag_completion_tool
  - accept_completion_tool
Agent ready! Type your questions below.
Enter 'quit' to stop.



You:  How does Qiskit's transpiler work?



Assistant: Qiskit's transpiler is a tool that optimizes quantum circuits for specific device topology. It takes a quantum circuit as input and generates an equivalent circuit that can be executed on a particular quantum computer or simulator.

The transpiler works by analyzing the quantum circuit and identifying opportunities to reduce the number of qubits, gates, and other resources required to execute the circuit. This is done using various optimization techniques, such as:

1. Qubit mapping: The transpiler maps the qubits in the original circuit to a smaller set of qubits that can be executed on the target device.
2. Gate selection: The transpiler selects the most efficient gates to use for each operation in the circuit.
3. Circuit simplification: The transpiler simplifies the circuit by removing unnecessary operations and combining similar gates.

The transpiler is an important component of Qiskit, as it allows users to write quantum circuits that can be executed on a variety of d

You:  quit


Goodbye!


## Custom Queries

Use this cell to ask the agent any question:

In [13]:
# Enter your question here:
MY_QUESTION = "Write a circuit with 3 qubits that applies a Toffoli gate"

async with mcp_client.session("qiskit-code-assistant") as session:
    agent = await create_agent_with_session(session)
    response = await ask_agent(agent, MY_QUESTION)
    print(response)

Loaded 4 tools from MCP server:
  - accept_model_disclaimer_tool
  - get_completion_tool
  - get_rag_completion_tool
  - accept_completion_tool
Here is a Qiskit circuit with 3 qubits that applies a Toffoli (CCX) gate:

```python
from qiskit import QuantumCircuit

# Create a circuit with 3 qubits
qc = QuantumCircuit(3)

# Apply a Toffoli (CCX) gate
qc.ccx(0, 1, 2)

# Draw the circuit
qc.draw('mpl')
```

This circuit applies a Toffoli gate with control qubits 0 and 1, and target qubit 2. The Toffoli gate flips the target qubit if and only if both control qubits are in the |1⟩ state.


## Available Tools

The agent has access to these tools provided by the MCP server:

| Tool | Description |
|------|-------------|
| `get_completion_tool` | Generate Qiskit code completions based on prompts |
| `get_rag_completion_tool` | Answer questions about Qiskit and quantum computing using RAG |
| `accept_model_disclaimer_tool` | Accept the disclaimer for a model |
| `accept_completion_tool` | Accept/acknowledge a generated completion |

## Troubleshooting

### Authentication errors?
- Verify your `QISKIT_IBM_TOKEN` is correct
- Make sure you have access to Qiskit Code Assistant
- Get your token from: https://quantum.ibm.com/

### MCP server not found?
Make sure `qiskit-code-assistant-mcp-server` is installed: `pip install qiskit-code-assistant-mcp-server[examples]`

### Model disclaimer required?
The agent will automatically use `accept_model_disclaimer_tool` when needed.